In [232]:
import regex as re

In [233]:
input_file = "input.py"

with open(input_file) as f:
    text = f.read()

print(text)

from highlighting_functions import Highlighter
import json as jorre

x = 37
this_variable = "hej på dig"
trippeljävel = """w"t"f"""

a_longer123string = (this_variable + 'ööhö')


In [234]:
def tokenize(text) -> list[str]:
    """Tokenize a text"""
    # Possible string delimiters
    string_delims = {'"', "'", '"""', "'''"}

    # split everything
    tokens = re.findall(r"(\w+|[^\w\s]+|\s+|\n)", text)

    # split tokens that doesn't contain letters?

    # TODO: doesn't split after last string

    # reunite string literals
    tokens_merged = []
    tags = []
    current_delim = ""
    current_string = ""
    strs_count = 0
    for t in tokens:
        if current_delim == "":
            if t in string_delims:
                # start a string
                strs_count += 1
                current_delim = t
                current_string += t
            else:
                tokens_merged.append(t)
                tags.append("<unk>")

        else:
            if t == current_delim:
                # break current string
                current_delim = ""
                current_string += t
                tokens_merged.append(current_string)
                tags.append("str")
                current_string = ""
            else:
                # add to current string
                current_string += t

    print(tokens)
    return tokens_merged, tags


tokens, tags = tokenize(text)
print(tokens)

['from', ' ', 'highlighting_functions', ' ', 'import', ' ', 'Highlighter', '\n', 'import', ' ', 'json', ' ', 'as', ' ', 'jorre', '\n\n', 'x', ' ', '=', ' ', '37', '\n', 'this_variable', ' ', '=', ' ', '"', 'hej', ' ', 'på', ' ', 'dig', '"', '\n', 'trippeljävel', ' ', '=', ' ', '"""', 'w', '"', 't', '"', 'f', '"""', '\n\n', 'a_longer123string', ' ', '=', ' ', '(', 'this_variable', ' ', '+', ' ', "'", 'ööhö', "')"]
['from', ' ', 'highlighting_functions', ' ', 'import', ' ', 'Highlighter', '\n', 'import', ' ', 'json', ' ', 'as', ' ', 'jorre', '\n\n', 'x', ' ', '=', ' ', '37', '\n', 'this_variable', ' ', '=', ' ', '"hej på dig"', '\n', 'trippeljävel', ' ', '=', ' ', '"""w"t"f"""', '\n\n', 'a_longer123string', ' ', '=', ' ', '(', 'this_variable', ' ', '+', ' ']


## tagging

In [235]:
print("# tokens", len(tokens))

known_chars = {
    "op": "=!%&/+-*",
    "brac_op": "([{",
    "brac_cl": ")]}",
}

# first: tag individual tokens

for i, token in enumerate(tokens):
    if token.isdigit():
        tags[i] = "num"
        continue
    for key in known_chars.keys():
        if token in known_chars[key]:
            tags[i] = key
            break


# print
for x in zip(tokens, tags):
    if not x[1] == "<unk>":
        print(*x)

# second: tag based on context


# tokens 43
= op
37 num
= op
"hej på dig" str
= op
"""w"t"f""" str
= op
( brac_op
+ op


In [236]:
print(tokens)

['from', ' ', 'highlighting_functions', ' ', 'import', ' ', 'Highlighter', '\n', 'import', ' ', 'json', ' ', 'as', ' ', 'jorre', '\n\n', 'x', ' ', '=', ' ', '37', '\n', 'this_variable', ' ', '=', ' ', '"hej på dig"', '\n', 'trippeljävel', ' ', '=', ' ', '"""w"t"f"""', '\n\n', 'a_longer123string', ' ', '=', ' ', '(', 'this_variable', ' ', '+', ' ']


In [237]:
classes = ("var", "op", "num")
